### rmsle

In [1]:
def rmsle(y, pred):
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(pred), 2)))

### 把随机森林的特征重要程度和特征名称重新组合成一个新的DataFrame

In [ ]:
col_ = pd.DataFrame({'importance':model.feature_importances_ , 'feature':col})
col_ = col_.sort_values(by=['importance'], ascending=[False])
col_

### 把测试集和训练集合并、分开

In [ ]:
df = pd.concat([training, test], axis=0)

sklearn里调节超参数，把超参数写成一个字典的形式，调好一部分后，然后再往里面加新的超参数，没加之前用固定的。这种贪心的策略能达到全局最优吗？

### 调参结果的保存

In [ ]:
#把lgb结果都放在一个DataFrame里
results = pd.DataFrame(columns = ["Rounds","Score","STDV", "LB", "Parameters"])
#各种参数的结果都放在一起，这里也保存了Rounds，训练轮数这个参数，调参完毕后
final_model_params = results.iloc[results['Score'].idxmin(),:]['Parameters']
optimal_rounds = results.iloc[results['Score'].idxmin(),:]['Rounds']
#选出最好的训练轮数和参数组合，优雅

### 训练好的模型，用不同的seed去预测

In [ ]:
all_seed = [27,22,300,401,7]
for seed in all_seed:
    final_model_params['seed'] = seed
    lgb_reg = lgb.train(
    final_model_params,
    lgtrain,
    num_boost_round = optimal_rounds + 1,
    verbose_eval=200)
        
    fold_importance_df = pd.DataFrame()
    fold_importance_df['feature'] = training.columns
    fold_importance_df['importance'] = lgb_reg.feature_importance()
    all_feature_importance_df = pd.concat([all_feature_importance_df,fold_importance_df], axis=0)
    
    muti_seed_pred[seed] = list(lgb_reg.predict(test))
    
    sub_preds = pd.DataFrame.from_dict(muti_seed_pred).replace(0,0.000001)
    
    lgb_ans = np.expm1(sub_preds.mean(axis=1))#这里有一个取指数的过程，因为之前去了对数；axis=1 对行取平均值

### k折交叉验证

可能有些模型不方便使用cress_predict，所以自己实现了一个k折交叉验证

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])

for n_fold ,(trn_idx,val_idx) in enumerate(folds.split(train)):
    trn_x,trn_y = train.ix[trn_idx],target[trn_idx]
    val_x,val_y = train.ix[val_idx], target[val_idx]
    cb_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=4, l2_leaf_reg=20, bootstrap_type='Bernoulli', subsample=0.6, eval_metric='RMSE', metric_period=50, od_type='Iter', od_wait=45, random_seed=17, allow_writing_files=False)
    cb_model.fit(trn_x, trn_y, eval_set=(val_x, val_y), cat_features=[], use_best_model=True, verbose=True)
    
    oof_preds[val_idx] = cb_model.predict(val_x)
    sub_preds += cb_model.predict(test) / folds.n_splits

### 对数转换后的结果处理

如果预测值的范围变化较大，会适应log转换把结果缩放到一个较小的范围内去拟合，但是最终提交结果的时候，要变换回去

In [ ]:
cb_ans = np.expm1(sub_preds)

### 如果某个特征取值只有一个，删除

In [ ]:
cols_with_onlyone_val = train.columns[train.nunique() == 1]

### 比较一个数据集的全部特征，如果某两个特征的值完全一样，删除

In [ ]:
colsToRemove = []
columns = train.columns
for i in range(len(columns)-1):
    v = train[columns[i]].values
    dupCols = []
    for j in range(i + 1,len(columns)):
        if np.array_equal(v, train[columns[j]].values):
            colsToRemove.append(columns[j])